# AuxTel - Sky Flats Sequence

This notebook is a draft of procedures we can use during twilight to take Sky Flats using different filters as per SITCOM-790. 
https://jira.lsstcorp.org/browse/SITCOM-790https://jira.lsstcorp.org/browse/SITCOM-790

It is meant to be executed between sunset and the time between the end of the civil and nautical twilights, that is, when the Sun's elevation is between 0 and 9 degrees below horizon. At 9 degrees below horizon, AuxTel should go on-sky to absorb zero-point offsets and perform the first WEP mirror alignment and focus of the night.

This procedure requires that AuxTel is ready for on-sky in advance, when the Sun is 25 degrees above horizon. 

Be prepared to run this notebook at the time of sunset to have enough time for the three available filters. 

After the setup is completed, we find an empty field in the opposite direction of the Sun, then we slew to this field, and take an image. This image will probably have saturation. We start a loop of taking images in the same position until we get an image that does not saturate. Also start with the bluest filter. 

Once the image is not saturated anymore, we take a sequence of images with small dithers and increasing exposure times. SITCOM-something like 12 flats per filter.

## Setup

In [1]:
import asyncio
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import logging
import time

from astropy import coordinates
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy import units as u
from astropy.time import Time
from astroquery.exceptions import NoResultsWarning
from astroquery.vizier import Vizier
from datetime import datetime
from warnings import warn

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.observing.utilities.auxtel.latiss.getters import get_image
# from lsst.ts.observing.utilities.auxtel.latiss.getters import get_image_sync as get_image

from lsst.summit.utils import BestEffortIsr

import lsst.daf.butler.cli.cliLog as cliLog
import lsst.daf.butler as dafButler

/net/obs-env/auto_base_packages/summit_extras/python/lsst/summit/__init__.py:23: FutureWarning: lsstimport has been a no-op since v19.0 and will be removed after v27.0. Please remove the import from your __init__.py.
  import lsstimport
/net/obs-env/auto_base_packages/ts_observatory_control/python/lsst/ts/observatory/control/utils/extras/dm_target_catalog.py:44: UserWarning: Cannot import required libraries. Module will not work.
  warnings.warn("Cannot import required libraries. Module will not work.")


In [2]:
logger = logging.getLogger(f"Sky Flats {Time.now()} UT")
logger.level = logging.INFO

In [3]:
# Instantiate the control classes
domain = salobj.Domain()
atcs = ATCS(domain)
atcs.set_rem_loglevel(logging.INFO)
latiss = LATISS(domain)
latiss.set_rem_loglevel(logging.INFO)
await asyncio.gather(atcs.start_task, latiss.start_task)

[[None, None, None, None, None, None, None], [None, None, None, None]]

In [4]:
inst_setup = await latiss.get_available_instrument_setup()
logger.info(f'LATISS filters are: {inst_setup[0]}')
logger.info(f'LATISS gratings are: {inst_setup[1]}')

In [5]:
repo = '/repo/LATISS/'
collection = 'LATISS/raw/all'

butler = dafButler.Butler(repo, collections=collection)
best_effort_isr = BestEffortIsr()

## Helper functions 

In [6]:
def get_target_radec(distance_from_sun=180, target_el=45, time=None):   
    """
    Returns the RADEC of the target area of the sky that's an azimuth `distance_from_sun` away from the Sun, given `elevation`,
    and at a given `time`.
    
    Parameters
    ----------
    distance_from_sun : float, (-180, 180)
        The distance from the Sun in degrees. Positive angles go towards the 
        North.
    elevation : float
        Target elevation for Sky Flats.
    time : datetime
        The time for the calculation in UTC.
    """
    
    az_sun, el_sun = atcs.get_sun_azel()
    
    target_az = (az_sun + distance_from_sun) % 360
    
    target_radec = atcs.radec_from_azel(target_az, target_el)
    
    return target_radec

In [7]:
def get_empty_field(target, radius=5):
    """
    Query the "Deep blank field catalogue : J/MNRAS/427/679" in Vizier.
    
    Parameters
    ----------
    target : astropy.coordinates.SkyCoord
        Sky coordinates near the field
    radius : float
        Search radius in degrees.
    
    Reference
    ---------
    http://cdsarc.u-strasbg.fr/viz-bin/Cat?J/MNRAS/427/679
    """
    _table = Vizier.query_region(
        catalog='J/MNRAS/427/679/blank_fld', 
        coordinates=target, 
        radius=radius*u.deg)

    if len(_table) == 0:
        warn(f"Could not find a field near {target} "
              f"within {radius} deg radius", category=NoResultsWarning)
        return None

    _table = _table["J/MNRAS/427/679/blank_fld"]
    
    coords = coordinates.SkyCoord(
        ra=_table["RAJ2000"], 
        dec=_table["DEJ2000"], 
        unit=(u.hourangle, u.deg), 
        frame=coordinates.ICRS)
    
    arg = target.separation(coords).argmin()
    
    return coords[arg]

In [8]:
async def take_triplets_with_fixed_dither(time, filt, offset):
    # await atcs.offset_xy(x=0, y=0)
    
    first_flat = await latiss.take_flats(
        exptime=time, nflats=1, filter=filt, grating='empty_1', reason="Sky_Flat", program="SITCOM-790")
    
    await atcs.offset_xy(x=offset, y=0)
    
    second_flat = await latiss.take_flats(
        exptime=time, nflats=1, filter=filt, grating='empty_1', reason="Sky_Flat", program="SITCOM-790")
    
    await atcs.offset_xy(x=0, y=offset)
    
    sky_counts = await latiss.take_flats(
        exptime=time, nflats=1, filter=filt, grating='empty_1', reason="Sky_Flat", program="SITCOM-790")
    
    logger.info(f"Sky flats triplets with {time} seconds exposure time and filter {filt} are {first_flat}, {second_flat} and {sky_counts}")
    

In [45]:
STD_TIMEOUT = 10  # seconds

async def verify_counts(exposure, timeout=STD_TIMEOUT, loop_time=1):
    """
    Retrieve image from butler repository.
    If not present, then it will poll at intervals of loop_time (0.1s default)
    until the image arrives, or until the timeout is reached.

    Parameters
    ----------
    exp: `float`
        exposure number, as 2023041500020 
    
    loop_time: `float`
        Time between polling attempts. Defaults to 0.1s
    timeout: `float`
        Total time to poll for image before raising an exception

    Returns
    -------

    median: `float`
        Exposure counts median across the detector
    """
    
    dataId = {'detector': 0, 'exposure': exposure}

    exp = await get_image(dataId, best_effort_isr, timeout = timeout)
    
    # Measure median, mean and std across the image. 
    foo = exp.getMaskedImage()
    masked_array = np.ma.masked_array(foo.image.array, mask=foo.mask.array)

    median = np.ma.median(masked_array)

    logger.info(f"Exposure ID: {exposure}")
    logger.info(f"    Median: {np.ma.median(masked_array)}")
    logger.info(f"    Mean: {np.ma.mean(masked_array)}")
    logger.info(f"    Std: {np.ma.std(masked_array)}")

    return median   

## Confirm Sun's elevation is fine to start taking Sky Flats. 

Check if we can start taking sky flats, that, if the Sun's elevation is between 0 and 9 degrees below the horizon.

In [13]:
sun_coordinates = atcs.get_sun_azel()

where_sun = "setting" if (sun_coordinates[0] > 180) else "rising"

# Print results
print(f" The azimuth of the {where_sun} Sun is {sun_coordinates[0]:.2f} deg \n"
      f" The elevation of the Sun is {sun_coordinates[1]:.2f} deg")

 The azimuth of the setting Sun is 280.07 deg 
 The elevation of the Sun is -25.48 deg


Only proceed with the next steps if the elevation of the Sun is between 0 and 9 degrees below the horizon, either rising or setting. 

## Find Field

Find a field with not many stars oppposite the Sun. 

In [18]:
target = get_target_radec()
search_area_degrees = 5
try: 
    empty_field_coords = get_empty_field(target, radius=search_area_degrees)
    logger.info(f"ICRS Empty field coordinates:\n"
                f"  RA  = {empty_field_coords.ra.to_string(u.hour, sep=':')} ;"
                f" DEC = {empty_field_coords.dec.to_string(u.degree, alwayssign=True, sep=':')}")
except:
    logger.info("No empty field was found in the catalog near your target area. \n"
                "Try increasing the search area in the variable search_area_degrees")

    (221.66725429, -26.99024035)> within 5 deg radius [warnings]


## Slew to Field

Now that we have our field, let's point the telescope and the dome to that direction.

In [19]:
empty_field_coords_azel = atcs.azel_from_radec(empty_field_coords.ra, empty_field_coords.dec)
logger.info(f"The telescope will be pointed to the empty field AzAlt coordinates:\n"
            f" AZ = {empty_field_coords_azel.az:.2f} deg ;"
            f" EL = {empty_field_coords_azel.alt:.2f} deg")

AttributeError: 'NoneType' object has no attribute 'ra'

In [17]:
await atcs.slew_icrs(empty_field_coords.ra, empty_field_coords.dec)

(<ICRS Coordinate: (ra, dec) in deg
     (196.838, -30.07411111)>,
 <Angle 0. deg>)

## Define filter, minimum counts and offsets between images

Need at least 10K ADU per pixel for each band.

In [38]:
order = [1,2,3,0]
filters = np.array(inst_setup[0])[order]

In [39]:
dither_offset = 30
min_counts_above_bias = 15000
multiple_factor = 1.5 

In [41]:
# !!!! Change this with Erik 
starting_exposure_time = {filters[0]:1, filters[1]:2, filters[2]:2, filters[3]:4}

In [21]:
newline = "\n"  # \escapes are not allowed inside f-strings
logger.info(f'The starting exposure time for each filter is \n'
            f'{newline.join(f"{key}: {value} seconds" for key, value in starting_exposure_time.items())}')

## Find first image

Here is where we check image saturation. We take an image with some short exposure time with the bluest filter, check how many pixels on each detector are saturated, check how many counts do we have in each detector, and once we have high but not saturated signal is all detectors and in most of the pixels, we move forward.

In [32]:
# Take image
test_exp = await latiss.take_flats(
    exptime=1, nflats=1, filter=filters[0], grating='empty_1', reason="test_saturation", program="SITCOM-790")

print(f"The test exposure is {test_exp[0]} with {filters[0]}")

signal_level = await verify_counts(test_exp[0], timeout=10)

The test exposure is 2023052300051 with SDSSg_65mm


AttributeError: 'coroutine' object has no attribute 'getMaskedImage'

## Start Sequence

Now that we have our start point for the bluest filter, we take triplets of sky flats with offsets of 30 arcsec between each. 

Confirm signal levels as they show up in RubinTv. 

In [ ]:
exposure_time = starting_exposure_time[filters[0]]

for filter_to_use in filters:
    logger.info(f"\n\n Filter is now {filter_to_use} \n\n")    
    
    flats_done = False
    max_exposure_time = 30 
    
    while not flats_done:
        background_level_exposure = await latiss.take_flats(
            exptime=exposure_time, nflats=1, filter=filter_to_use, grating='empty_1', reason="test_background_level", program="SITCOM-790")

        background_level = await verify_counts(background_level_exposure) 
        
        if background_level > min_counts_above_bias:
            await take_triplets_with_fixed_dither(exposure_time, filter_to_use, dither_offset)            
            logger.info("Changing filters")
            flats_done = True

        else:
            exposure_time = exposure_time * (min_counts_above_bias/background_level) * multiple_factor
            logger.info(f"The exposure time will be increased to {exposure_time} seconds")

            if exposure_time > max_exposure_time:         
                logger.info(f"Sky background in {filter_to_use} too low")
                logger.info("Changing filters")
                flats_done = True           

## Image Assessment

The two next cells will serve to check the median, mean and std of the signal across an image. 
Edit the `day_obs` and `seq_num` in the `dataId` row. 

In [ ]:
dataId = {'detector': 0, 'day_obs': 20230427, 'seq_num': 87}
exp = butler.get('raw', dataId=dataId)

In [ ]:
foo = exp.getMaskedImage()
masked_array = np.ma.masked_array(foo.image.array, mask=foo.mask.array)
meta = exp.getMetadata()

logger.info(f"Observation ID: {meta['OBSID']}")
logger.info(f"    Median: {np.ma.median(masked_array)}")
logger.info(f"    Mean: {np.ma.mean(masked_array)}")
logger.info(f"    Std: {np.ma.std(masked_array)}")